# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

In [ ]:
import xlwings as xw
import numpy as np
import pandas as pd
from xlwings import constants 
wb = xw.Book('себестоимостьА_в1.xlsx')
recipies = wb.sheets['Рецептура']

ModuleNotFoundError: ignored

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

In [ ]:
consumption = recipies.range('G7:O10').options(np.array).value
consumption = np.nan_to_num(consumption)
unit_prices = recipies.range('G14:O14').options(np.array).value
cost = consumption @ unit_prices
cost

array([21.48 , 16.525, 17.423, 18.085])

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

In [ ]:
recipies.range('T6').value = 'Себестоимость'
recipies.range('T7').options(transpose=True).value = cost

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

In [ ]:
recipies.range('S3:S16').copy()
recipies.range('T3:T16').paste('formats')
recipies.range('T4').value = 'Себестоимость'
xw.Range('T4:T6').autofit()
xw.Range('T7:T10').number_format = '0,00'

4. Выполнить 3 с помощью "протягиваемых" формул.

In [ ]:
xw.Range('T7').formula = '=SUMPRODUCT(G7:O7,$G$14:$O$14)' 
xw.Range('T7:T10').formula = xw.Range('T7').formula

## Лабораторная работа 7.1

In [ ]:
import numpy as np
import pandas as pd

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [ ]:
reviews = pd.read_csv("reviews_sample.csv", index_col=0)
reviews.head()

,user_id,recipe_id,date,rating,review
370476,21752,57993,2003-05-01,5,Last week whole sides of frozen salmon fillet ...
624300,431813,142201,2007-09-16,5,So simple and so tasty! I used a yellow capsi...
187037,400708,252013,2008-01-10,4,"Very nice breakfast HH, easy to make and yummy..."
706134,2001852463,404716,2017-12-11,5,These are a favorite for the holidays and so e...
312179,95810,129396,2008-03-14,5,Excellent soup! The tomato flavor is just gre...


In [ ]:
recipes = pd.read_csv(
    "recipes_sample.csv",
    usecols=["id","name","minutes", "submitted", "description", "n_ingredients"],
    index_col="id"
)
recipes.head()

,name,minutes,submitted,description,n_ingredients
id,,,,,
44123,george s at the cove black bean soup,90,2002-10-25,an original recipe created by chef scott meska...,18.0
67664,healthy for them yogurt popsicles,10,2003-07-26,my children and their friends ask for my homem...,NaN
38798,i can t believe it s spinach,30,2002-08-29,"these were so go, it surprised even me.",8.0
35173,italian gut busters,45,2002-07-27,my sister-in-law made these for us at a family...,NaN
84797,love is in the air beef fondue sauces,25,2004-02-23,i think a fondue is a very romantic casual din...,NaN


2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [ ]:
recipes_sample = recipes.sample(frac=0.05)

In [ ]:
reviews_sample = reviews.sample(frac=0.05)

In [ ]:
with pd.ExcelWriter("my_file.xlsx") as writer:
    recipes_sample.to_excel(writer, sheet_name="Рецепты")
    reviews_sample.to_excel(writer, sheet_name="Отзывы")

3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [ ]:
import xlwings as xw
from xlwings import constants 
wb = xw.Book('my_file.xlsx')

In [ ]:
for sh in wb.sheets:
    if sh.name=="Отзывы":
        reviews_sh = sh
    else:
        recipes_sh = sh

In [ ]:
last_col = recipes_sh.range((1,1)).expand("right").last_cell.column
seconds = (recipes_sample[["minutes"]] * 60)
seconds.columns = ["seconds_assign"]
recipes_sh.range((1,last_col+1)).options(pd.DataFrame, index=False).value = seconds

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [ ]:
last_col = recipes_sh.range((1,1)).expand("right").last_cell.column
last_row = recipes_sh.range((1,1)).expand("down").last_cell.row
recipes_sh.range((1,last_col+1)).value = "seconds_formula"
addr = recipes_sh.range((2,3)).get_address(row_absolute=False, column_absolute=False)

recipes_sh.range((2,last_col+1),(last_row,last_col+1)).formula = f"={addr}*60"

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [ ]:
columns_range = recipes_sh.range((1,last_col)).expand("right")
columns_range.font.bold = True
columns_range.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter
columns_range.api.VerticalAlignment = xw.constants.VAlign.xlVAlignCenter

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [ ]:
for cell in recipes_sh.range((2,3)).expand("down"):
    if cell.value<5:
        cell.color = "#008000"
    elif cell.value<10:
        cell.color = "#ffff00"
    else:
        cell.color = "ff0000"

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [ ]:
last_col = recipes_sh.range((1,1)).expand("right").last_cell.column
addr = reviews_sh.range((1,3)).expand("down").get_address(include_sheetname=True)
addr2 = recipes_sh.range((2,1)).get_address(row_absolute=False, column_absolute=False)
recipes_sh.range((1,last_col+1)).value = "n_reviews"
recipes_sh.range((2,last_col+1),(last_row,last_col+1)).formula = f"=COUNTIF({addr},{addr2})"

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [ ]:
def validate(reviews_sh, recipes_sh):
    rating_col = reviews_sh.range((1,1)).expand("right").value.index("rating")
    rec_id_col = reviews_sh.range((1,1)).expand("right").value.index("recipe_id")
    id_col = recipes_sh.range((1,1)).expand("right").value.index("id")
    ids = recipes_sh.range((2,id_col+1)).expand("down").value

    for row in reviews_sh.range((2,1)).expand("table").rows:
    #     print(row[rec_id_col],)
        if not (0<=int(row[rating_col].value)<=5):
            row.color = "ff0000"  
        elif not row[rec_id_col].value in ids:
    #         print(row[rec_id_col])
            row.color = "#ffff00"

In [ ]:
validate(reviews_sh, recipes_sh)

9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [ ]:
import csv

In [ ]:
with open("recipes_model.csv", "r", encoding="utf8") as file:
    reader = csv.reader(file, delimiter="\t")
    table = [row for row in reader]
    
wb = xw.Book()
model_sh = wb.sheets[0]
model_sh.name = "Модель"
model_sh.range((2,1)).value = table
wb.save("recipes_model.xlsx")

![image.png](attachment:image.png)

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
last_row = len(table)+1
last_col = len(table[0])+1

In [ ]:
addrB = model_sh.range((2,2)).get_address(row_absolute=False, column_absolute=False)
addrC = model_sh.range((2,3)).get_address(row_absolute=False, column_absolute=False)
addrD = model_sh.range((2,4)).get_address(row_absolute=False, column_absolute=False)
addrG = model_sh.range((2,7)).get_address(row_absolute=False, column_absolute=False)
addrH = model_sh.range((2,8)).get_address(row_absolute=False, column_absolute=False)
addrI = model_sh.range((2,9)).get_address(row_absolute=False, column_absolute=False)
concat= f"""CONCATENATE(" REFERENCES ", {addrH},"(",{addrI},")")"""
switch = f"""SWITCH({addrG}, "PK"," PRIMARY KEY", "FK", {concat},"")"""
if_ = f"""IF(AND({addrD}="Y",{addrG}<>"PK")," NOT NULL", "")"""
formula = f"""=CONCATENATE({addrB}," ", {addrC}, " ",{switch},{if_})"""
formula
model_sh.range((2,10),(last_row,10)).formula = formula

![image.png](attachment:image.png)

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [ ]:
tbl = model_sh.tables.add(model_sh.range((1,1),(last_row,last_col)), table_style_name='TableStyleMedium6') 

In [ ]:
tbl.header_row_range.color = "#00ccff"
tbl.header_row_range.value = [
    "table_name",
    "attr_name",
    "type",
    "NOT_NULL",
    "rus_table_name",
    "rus_attr_name",
    "key",
    "ref_table",
    "ref_attr",
    "attr_generation"
]
tbl.range.autofit()
tbl.header_row_range.font.bold = True

![image.png](attachment:image.png)

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 

In [ ]:
tbl_frame = tbl.range.options(pd.DataFrame, index=False).value
tbl_frame.head()

,table_name,attr_name,type,NOT_NULL,rus_table_name,rus_attr_name,key,ref_table,ref_attr,attr_generation
0,recipe,name,varchar(100),Y,Рецепт,Название рецепта,None,None,None,name varchar(100) NOT NULL
1,recipe,id,serial,N,Рецепт,ID рецепта,PK,None,None,id serial PRIMARY KEY
2,recipe,minutes,int,Y,Рецепт,Длительность выполнения в минутах,None,None,None,minutes int NOT NULL
3,recipe,contributor_id,int,Y,Рецепт,ID автора рецепта,FK,user,id,contributor_id int REFERENCES user(id) NOT NULL
4,recipe,submitted,timestamp,Y,Рецепт,Дата добавления,None,None,None,submitted timestamp NOT NULL


In [ ]:
entity = tbl_frame.groupby("table_name").count()["attr_name"]

In [ ]:
statistic_sh = wb.sheets.add("Статистика")

In [ ]:
statistic_sh.range((1,1)).options(pd.Series, header=False).value = entity

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
plt.bar(entity.index, entity)
plt.xlabel('table name')
plt.ylabel('attr count')

my_pict = statistic_sh.pictures.add(
    fig, name='MyPlot', update=True,
    left=statistic_sh.range('E2').left,
    top=statistic_sh.range('E2').top
)   

![image.png](attachment:image.png)